In [1]:
import openseespy.opensees as ops
import opsvis as opsv
import pystra as ra
import matplotlib.pyplot as plt
import numpy as np

In [6]:
def single_run(E=30e3,P=25.0,w=0.1,x=0.0):
    ops.wipe()
    ops.model('basic', '-ndm', 2, '-ndf', 3)

    ops.node(1, x, 0)
    ops.node(2, 0, 144)
    ops.node(3, 240, 144)
    ops.node(4, 240, 0)

    ops.fix(1, 1, 1, 1)
    ops.fix(4, 1, 1, 1)

    Ag = 25.0
    Ig = 1500.0
    Ac = 29.0
    Ic = 2000.0

    gsecTag = 1
    ops.section("Elastic", gsecTag, E, Ag, Ig)

    csecTag = 2
    ops.section("Elastic", csecTag, E, Ac, Ic)

    transfTag = 1
    ops.geomTransf("Linear", transfTag)

    N = 3

    gbiTag = 1
    ops.beamIntegration("Lobatto", gbiTag, gsecTag, N)
    cbiTag = 2
    ops.beamIntegration("Lobatto", cbiTag, csecTag, N)

    leftColTag = 1
    ops.element("forceBeamColumn", leftColTag, 1, 2, transfTag, cbiTag)
    girderTag = 2
    ops.element("forceBeamColumn", girderTag, 2, 3, transfTag, gbiTag)
    rightColTag = 3
    ops.element("forceBeamColumn", rightColTag, 3, 4, transfTag, cbiTag)

    tsTag = 1
    ops.timeSeries("Constant", tsTag)

    patternTag = 1
    ops.pattern("Plain", patternTag, tsTag)

    ops.load(2, P, 0, 0)
    ops.eleLoad("-ele", girderTag, "-type", "beamUniform", -w)

    # define these to avoid warnings
    ops.constraints('Transformation')
    ops.numberer('RCM')
    ops.system('BandGeneral')
    ops.test('NormDispIncr', 1.0e-6, 6, 2)
    ops.algorithm('Linear')
    ops.integrator('LoadControl', 1)
    ops.analysis('Static')
    ops.analyze(1);
    
    return 0.15 - ops.nodeDisp(2,1)

In [7]:
def lsf(E,P,w,x):
    n = len(E)
    g = np.zeros((n,1))
    
    for i in range(n):
        g[i] = single_run(E[i],P[i],w[i],x[i])
    
    return g.T

In [8]:
def plot_ops_model_results():
    opsv.plot_model();
    opsv.plot_loads_2d(sfac=5)
    opsv.plot_defo()
    sfacN, sfacV, sfacM = 2, 2, 5.e-2
    opsv.section_force_diagram_2d('N', sfacN)
    plt.title('Axial force distribution')
    opsv.section_force_diagram_2d('T', sfacV)
    plt.title('Shear force distribution')
    opsv.section_force_diagram_2d('M', sfacM)
    plt.title('Bending moment distribution')
    plt.show()

In [10]:
E=30e3
P=25.0
w=0.1
x=0.0

ops.randomVariable(62, 'lognormal', '-mean', E, '-stdv', 0.1*E)
ops.randomVariable(32, 'normal', '-mean', P, '-stdv', 0.2*P)
ops.randomVariable(89, 'normal', '-mean', 0, '-stdv', 1)
ops.randomVariable(41, 'normal', '-mean', -w, '-stdv', abs(0.2*w))

ops.parameter(12, 'randomVariable', 62, "element", leftColTag, "E")
ops.addToParameter(12, 'element', rightColTag, "E")
ops.parameter(25, 'randomVariable', 32,"loadPattern", patternTag,"loadAtNode", 2, 1)
ops.parameter(3, 'randomVariable', 89,"node", 1, "coord", 1)
ops.parameter(45, 'randomVariable', 41,"loadPattern", patternTag,"elementLoad", girderTag, "wy")
ops.parameter(23, "node", 2, "disp", 1)

ops.performanceFunction(76, "0.15-par[23]")

ops.sensitivityAlgorithm("-computeAtEachStep")

ops.randomNumberGenerator('CStdLib')
ops.probabilityTransformation('Nataf', "-print", 3)
ops.reliabilityConvergenceCheck('Standard', "-e1", 1e-2,
                                "-e2", 1e-2, "-print", 1)
ops.functionEvaluator('Python', "-file", "opensees.analyze(1)")
ops.gradientEvaluator('Implicit')
ops.searchDirection('iHLRF')
ops.meritFunctionCheck('AdkZhang', "-multi", 2.0, "-add", 50.0,
                       "-factor", 0.5)
ops.stepSizeRule('Armijo', "-maxNum", 10, "-base", 0.5,
                 "-initial", 0.3, 5)
ops.stepSizeRule('Fixed', "-stepSize", 1.0)
ops.startPoint('Mean')
ops.findDesignPoint('StepSearch', "-maxNumIter", 30)

ops.runFORMAnalysis('portalframe.out')

for perf in ops.getLSFTags():
    print(f"Performance Function {perf}")
    print(f"beta = {ops.betaFORM[perf]:.7f}")
    for rv in ops.getRVTags():
        print(
            f"\t x*({rv}) = {ops.designPointXFORM[perf, rv]:7.4f}, alpha({rv}) = {ops.alphaFORM[perf, rv]:7.4f}, gamma({rv}) = {ops.gammaFORM[perf, rv]}")

NameError: name 'leftColTag' is not defined

## Pystra

In [81]:
limit_state = ra.LimitState(lsf)
options = ra.AnalysisOptions()

In [84]:
stochastic_model = ra.StochasticModel()
stochastic_model.addVariable(ra.Lognormal("E", 30e3, 3e3))
stochastic_model.addVariable(ra.Normal("P", 25, 5))
stochastic_model.addVariable(ra.Normal("x", 0, 1))
stochastic_model.addVariable(ra.Uniform("w", 0.1, 0.02))

In [85]:
form = ra.Form(
    analysis_options=options,
    stochastic_model=stochastic_model,
    limit_state=limit_state,
)
form.run()
form.showDetailedOutput()


FORM
Pf              	 1.2910055315e-02
BetaHL          	 2.2289067392
Model Evaluations 	 71
------------------------------------------------------
Variable   	    U_star 	       X_star 	     alpha
E          	 -1.256322 	 26335.092481 	 -0.563616
P          	  1.841094 	    34.205468 	 +0.826030
x          	  0.007319 	     0.007319 	 +0.003282
w          	 -0.000060 	     0.099998 	 -0.000027



In [87]:
sorm = ra.Sorm(
    analysis_options=options,
    stochastic_model=stochastic_model,
    limit_state=limit_state,
    form=form,
)
sorm.run()
sorm.showDetailedOutput()


FORM/SORM
Pf FORM         		 1.2910055315e-02
Pf SORM Breitung 		 1.2392501137e-02
Pf SORM Breitung HR 	 1.2317169145e-02
Beta_HL         		 2.2289067392
Beta_G Breitung 		 2.2447372958
Beta_G Breitung HR 		 2.2470890234
Model Evaluations 		 121
------------------------------------------------------
Curvature 1: -0.0037714867634500203
Curvature 2: 0.003669292413446555
Curvature 3: 0.038401380765963646
------------------------------------------------------
Variable   	    U_star 	       X_star 	     alpha
E          	 -1.256322 	 26335.092481 	 -0.563616
P          	  1.841094 	    34.205468 	 +0.826030
x          	  0.007319 	     0.007319 	 +0.003282
w          	 -0.000060 	     0.099998 	 -0.000027

